In [262]:
import random
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout,AveragePooling2D
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score

In [263]:
class LayerFactory:
    def __init__(self):
        self.func_mapping = {
            0: 'get_random_conv_2d',
            1: 'get_random_max_pooling',
            2: 'get_random_avg_pooling',
            3: 'get_random_dropout'
        }
        self.num_of_layer_types = len(self.func_mapping)
        
    def get_random(self):
        layer_type = self.func_mapping[random.randrange(0, self.num_of_layer_types)]
        layer_func = getattr(self, layer_type)
        return layer_func()
        
    def get_random_conv_2d(self):
        return Conv2D(self.randomFiltersConv2D(), kernel_size = self.randomKernelSizeConv2D(), strides = self.randomStrideConv2D(), padding = self.randomPaddingConv2D(), activation = self.randomActivationConv2D())
    def get_random_max_pooling(self):
        return MaxPooling2D(pool_size = self.randomPoolSize())
    def get_random_avg_pooling(self):
        return AveragePooling2D(pool_size = self.randomPoolSize())
    def get_random_dropout(self):
        return Dropout(random.uniform(0,1))
    

    def randomFiltersConv2D(self):
        return random.choice([32, 64])
    
    def randomKernelSizeConv2D(self):
        return random.choice([(3, 3),(2,2)])
    
    def randomStrideConv2D(self):
        return random.choice([(1, 1),(2,2)])
    
    def randomPaddingConv2D(self):
        return random.choice(["same"])
    
    def randomActivationConv2D(self):
        return random.choice(["relu"])
    
    def randomPoolSize(self):
        return random.choice([(2,2)])

In [264]:
class Individual:
    def __init__(self, input_shape):
        self.code = []
        self.input_shape = input_shape
        self.num_middle_layers = 0
        self.fitness = 0

    
    def from_configured_layers(self, layers):
        self.code = [self._resolve_layer(layer) for layer in layers]
        
        self.num_middle_layers = len(layers)
        self.fitness = 0 #reset fitness just in case
                    
    def fill_random(self, num_middle_layers, lf):
        self.num_middle_layers = num_middle_layers 
        for i in range(self.num_middle_layers):
            self.code.append(lf.get_random())
            
    def calculate_fitness(self, X_train, y_train, X_validation, y_validation):
        
        model = self._to_model()
        
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        model.fit(X_train, y_train, batch_size=128, epochs=9, validation_split=0.2)
        model.evaluate(X_validation, y_validation)
        
        y_pred = np.argmax(model.predict(X_validation),axis = 1)
        y_true = np.argmax(y_validation, axis = 1)
        
        self.fitness = accuracy_score(y_true, y_pred)
        
    def __lt__(self, other):
        return self.fitness < other.fitness
        
    def __str__(self):
        modelString = ""
        for i in range(len(self.dictionary)):
            modelString+=" "
            modelString+=str(i)
        
        return str(self.dictionary)
            
    def _to_model(self):
        model = keras.Sequential()
        model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=self.input_shape))
        
        for layer in self.code:
            model.add(layer)
            
        model.add(keras.layers.Flatten())
        model.add(keras.layers.Dense(units = 10 ,activation = 'softmax'))
        
        return model
            
    def _resolve_layer(self, layer):
        layer_config = layer.get_config()
        if type(layer) == Conv2D:
            return Conv2D(filters = layer_config['filters'], kernel_size = layer_config['kernel_size'], strides = layer_config['strides'], padding = layer_config['padding'], activation = layer_config['activation'])
        elif type(layer) == MaxPooling2D:
            return MaxPooling2D(pool_size = layer_config['pool_size'])
        elif type(layer) == AveragePooling2D:
            return AveragePooling2D(pool_size = layer_config['pool_size'])
        elif type(layer) == Dropout:
            return Dropout(layer_config['rate'])
        else:
            print("Unsupported layer type", type(layer))
            return None

In [265]:
def selection(population):
    TOURNAMENT_SIZE = 5
    participants = random.sample(population, TOURNAMENT_SIZE)
    winner = max(participants)
    return winner

In [266]:
def mutation(individual, lf):
    MUTATION_PROB = 0.05
    for i in range(len(individual.code)):
        if random.random() < MUTATION_PROB:
            individual.code[i] = lf.get_random()

In [267]:
import copy
def crossover(parent1, parent2, child1, child2):
    num_of_layers_child1 = random.randrange(min(parent1.num_middle_layers, parent2.num_middle_layers), max(parent1.num_middle_layers, parent2.num_middle_layers) + 1)
    num_of_layers_child2 = random.randrange(min(parent1.num_middle_layers, parent2.num_middle_layers), max(parent1.num_middle_layers, parent2.num_middle_layers) + 1)
    
    children_middle_layers = parent1.code + parent2.code
    
    child1_layers = random.sample(children_middle_layers, num_of_layers_child1)
    child2_layers = random.sample(children_middle_layers, num_of_layers_child2)
    
    child1.from_configured_layers(child1_layers)
    child2.from_configured_layers(child2_layers)
    


In [268]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K

(X_train, y_train), (X_test, y_test) = mnist.load_data()
img_size = X_train.shape[1]
if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_size, img_size)
    X_test = X_test.reshape(X_test.shape[0], 1, img_size, img_size)
else:
    X_train = X_train.reshape(X_train.shape[0], img_size, img_size, 1)
    X_test = X_test.reshape(X_test.shape[0], img_size, img_size, 1)
X_train = X_train / 255

X_test = X_test / 255

X_train = X_train[:1000]
X_validation = X_test[550:1000]
X_test = X_test[:550]
num_classes = 10
y_train = to_categorical(y_train[:1000],num_classes)
y_validation = to_categorical(y_test[550:1000], num_classes)
y_test = to_categorical(y_test[:550], num_classes)

In [7]:
POP_SIZE = 14
NUM_GENERATIONS = 4
ELITISM_SIZE = POP_SIZE // 3
MIN_MID_LAYERS = 1
MAX_MID_LAYERS = 5
if ELITISM_SIZE % 2 == 1:
    ELITISM_SIZE -= 1

lf = LayerFactory()

population = [Individual((28, 28, 1)) for _ in range(POP_SIZE)]
newPopulation = [Individual((28, 28, 1)) for _ in range(POP_SIZE)]

for individual in population: 
    individual.fill_random(random.randrange(MIN_MID_LAYERS, MAX_MID_LAYERS), lf)
    individual.calculate_fitness(X_train, y_train, X_validation, y_validation)

In [4]:
experiment = 10
best_models = []

for k in range(experiment):
    for i in range(NUM_GENERATIONS):
        population.sort(reverse=True)
        if population[0].fitness >= 0.96:
            break

        newPopulation[:ELITISM_SIZE] = population[:ELITISM_SIZE]
        for j in range(ELITISM_SIZE, POP_SIZE, 2):
            parent1 = selection(population)
            parent2 = selection(population)

            crossover(parent1, parent2, newPopulation[j], newPopulation[j+1])

            mutation(newPopulation[j], lf)
            mutation(newPopulation[j+1], lf)

            newPopulation[j].calculate_fitness(X_train, y_train, X_validation, y_validation)
            newPopulation[j + 1].calculate_fitness(X_train, y_train, X_validation, y_validation)    

        population = newPopulation
    best = max(population)
    best_model = best._to_model()
    y_pred = np.argmax(best_model.predict(X_test),axis = 1)
    y_true = np.argmax(y_test, axis = 1)
    fitness = accuracy_score(y_true, y_pred)
    best_models.append((best_model,fitness))

In [6]:
for b in best_models:
    print(b[0].summary(),b[1])